In [1]:
import tensorflow as tf
import tensorflow.keras as keras

# model = keras.models.load_model("Mobilefacenet-TF2-coral_tpu/model/model_with_mask_clf.h5")
model = keras.models.load_model("Mobilefacenet-TF2-coral_tpu/pretrained_model/training_model/inference_model_0.993.h5")

In [2]:
from keras.models import Model
from tensorflow.python.keras.layers import Lambda;

# new_model =  Model(inputs = model.input, outputs = model.get_layer('model')(model.input))
# new_model.summary()
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('saved_models/lite/mobilefacenet.tflite', 'wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\GOKULA~1\AppData\Local\Temp\tmpaflgqlva\assets


INFO:tensorflow:Assets written to: C:\Users\GOKULA~1\AppData\Local\Temp\tmpaflgqlva\assets


In [3]:
import sys
import os
home_path = os.path.abspath("")

import warnings
warnings.filterwarnings('ignore')

# sys.path.append(home_path+'\\'+"seesaw-facenet")
# sys.path.append(home_path+'\\'+"seesawfacenet_pytorch")
# sys.path.append(home_path+'\\'+"seesawfacenet_pytorch\\src")

import torch
from torchinfo import summary
from torchvision.transforms import transforms as T

from seesawfacenet_pytorch.src.seesaw_models import seesaw_shareFaceNet, seesaw_shuffleFaceNet, DW_seesawFaceNetv2
from PIL import Image
import cv2
import numpy as np
import torch.nn as nn
from tqdm.auto import tqdm


In [4]:
class CustomNet(nn.Module):
    def __init__(self, model):
        super(CustomNet, self).__init__()
        self.pre_process = T.Compose([T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
        self.backbone = model

    def forward(self,x):
        x = self.pre_process(x)
        return self.backbone(x)


# class re_CustomNet(nn.Module):
#     def __init__(self, model):
#         super(CustomNet, self).__init__()
#         self.pre_process = T.Compose([T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
#         self.backbone = model

#     def forward(self,x):
#         x = self.pre_process(x)
#         return self.backbone(x)

reimplement_model_seesawnet = CustomNet(torch.jit.load("saved_models/Seesaw_shuffleFaceNet.pt")).cpu().eval()


In [5]:
def get_seesawnet_model(model, saved_dict):
    # saved_dict = torch.load("saved_models/Seesaw-shuffleFaceNet_160bs/save/model_2019-06-22-17-20_accuracy_0.9184285714285714_step_568064_final.pth", map_location="cuda")
    saved_dict = {v[7:]:k for v,k in saved_dict.items()}
    model.load_state_dict(saved_dict)
    model = model.cpu()
    model.eval()

    seesawnet_model = CustomNet(model)
    seesawnet_model.eval()

    img = Image.open("img.jpg")
    img = torch.FloatTensor(cv2.resize(np.asarray(img), (112, 112))).permute(2,1,0)
    img = img/255.
    img = img.unsqueeze(0)
    out = model(img)

    return seesawnet_model

In [6]:
from torch.utils.mobile_optimizer import optimize_for_mobile

models_configs = [("seesaw_shuffleFaceNet_160", seesaw_shuffleFaceNet.seesaw_shuffleFaceNet(512), "saved_models/Seesaw-shuffleFaceNet_160bs/save/model_2019-06-22-17-20_accuracy_0.9184285714285714_step_568064_final.pth"),
                 ("seesaw_shuffleFaceNet_192", seesaw_shuffleFaceNet.seesaw_shuffleFaceNet(512), "saved_models/Seesaw-shuffleFaceNet_192bs/save/model_2019-06-14-06-06_accuracy_0.9269999999999999_step_475328_final.pth"),
                 ("seesaw_shareFaceNet_160", seesaw_shareFaceNet.seesaw_shareFaceNet(512), "saved_models/Seesaw-shareFaceNet_160bs/save/model_2019-06-19-04-38_accuracy_0.9298571428571428_step_568064_final.pth"),
                 ("DW_seesawFaceNetv2", DW_seesawFaceNetv2.DW_seesawFaceNetv2(512), "saved_models/DW-SeesawFaceNetv2_128bs/save/model_2019-08-03-07-50_accuracy_0.9486000000000001_step_727840_final.pth"),
                 ("reimplementation_seesawnet",reimplement_model_seesawnet,None)
]

for name,model,saved_dict_path in tqdm(models_configs):
    if saved_dict_path == None:
        model = model

    else:
        model = get_seesawnets_model(model, torch.load(saved_dict_path))
        
    example = torch.rand(1, 3, 112, 112)

    traced_script_module = torch.jit.trace(model, example)
    traced_script_module_optimized = optimize_for_mobile(traced_script_module)
    traced_script_module_optimized._save_for_lite_interpreter(f"saved_models/lite/{name}.ptl")
    
    # m_jit = torch.jit.script(model)
    # m_jit.save(f"saved_models/jit/{name}.ptl")

100%|██████████| 5/5 [00:21<00:00,  4.23s/it]
